<a href="https://colab.research.google.com/github/souma21122002/Internship2025/blob/main/Celeb_DF_frame_face1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
zip_ref=zipfile.ZipFile('/content/drive/MyDrive/Internship2025/Dataset/celeb_DF.zip')
zip_ref.extractall('/content/drive/MyDrive/Internship2025/Dataset')
zip_ref.close()

In [1]:
!pip install mediapipe==0.10.20 --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 

In [ ]:
!pip install mediapipe==0.10.20
!pip install opencv-python

In [ ]:

import cv2
import mediapipe as mp
import os
from google.colab.patches import cv2_imshow
from google.colab import drive

# Mount Google Drive (added error handling)
try:
    drive.mount('/content/drive')
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    exit()

# Define paths
input_folder = '/content/drive/MyDrive/Internship2025/Dataset/Celeb-synthesis'
cropped_faces_folder = '/content/drive/MyDrive/Internship2025/Mediapipe_output_face_frame/Celeb_synthesis_mediapipe_output/faces_synthesis_mediapipe/'
original_frames_folder = '/content/drive/MyDrive/Internship2025/Mediapipe_output_face_frame/Celeb_synthesis_mediapipe_output/frames_synthesis_mediapipe/'

# Create output directories if they don't exist
os.makedirs(cropped_faces_folder, exist_ok=True)
os.makedirs(original_frames_folder, exist_ok=True)

# Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
DETECTION_CONFIDENCE = 0.5

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=DETECTION_CONFIDENCE) as face_detector:
    for video_file in os.listdir(input_folder):
        video_path = os.path.join(input_folder, video_file)

        if video_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
            print(f"Processing video: {video_file}")
            base_name = os.path.splitext(video_file)[0]

            # Create subfolders for each video
            video_cropped_folder = os.path.join(cropped_faces_folder, base_name)
            video_real_frame_folder = os.path.join(original_frames_folder, base_name)
            os.makedirs(video_cropped_folder, exist_ok=True)
            os.makedirs(video_real_frame_folder, exist_ok=True)

            cap = cv2.VideoCapture(video_path)
            frame_count = 0

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame_count += 1

                # Convert BGR to RGB for MediaPipe
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = face_detector.process(rgb_frame)

                if results.detections:
                    # Save the full frame (original) if face is detected
                    real_frame_filename = f"{base_name}_frame_{frame_count:06d}.jpg"
                    real_frame_path = os.path.join(video_real_frame_folder, real_frame_filename)
                    cv2.imwrite(real_frame_path, frame)

                    # Save cropped faces
                    for i, detection in enumerate(results.detections):
                        bboxC = detection.location_data.relative_bounding_box
                        ih, iw, _ = frame.shape
                        x1 = int(bboxC.xmin * iw)
                        y1 = int(bboxC.ymin * ih)
                        w = int(bboxC.width * iw)
                        h = int(bboxC.height * ih)
                        x2 = x1 + w
                        y2 = y1 + h

                        # Ensure coordinates are within frame bounds
                        cropped_face = frame[max(0, y1):min(ih, y2), max(0, x1):min(iw, x2)]

                        if cropped_face.shape[0] > 0 and cropped_face.shape[1] > 0:
                            cropped_filename = f"{base_name}_face_{frame_count:06d}.jpg"
                            cropped_path = os.path.join(video_cropped_folder, cropped_filename)
                            cv2.imwrite(cropped_path, cropped_face)

            cap.release()
            print(f"Finished processing: {video_file}")
            print(f"→ Real frames saved to: {video_real_frame_folder}")
            print(f"→ Cropped faces saved to: {video_cropped_folder}")

cv2.destroyAllWindows()

/usr/local/lib/python3.11/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.4.33 is installed, but it is not compatible with the installed jaxlib version 0.6.1, so it will not be used.
  warnings.warn(


Processing video: id28_id22_0000.mp4
Finished processing: id28_id22_0000.mp4
→ Real frames saved to: /content/drive/MyDrive/Internship2025/Mediapipe_output_face_frame/Celeb_synthesis_mediapipe_output/frames_synthesis_mediapipe/id28_id22_0000
→ Cropped faces saved to: /content/drive/MyDrive/Internship2025/Mediapipe_output_face_frame/Celeb_synthesis_mediapipe_output/faces_synthesis_mediapipe/id28_id22_0000
Processing video: id29_id21_0009.mp4
Finished processing: id29_id21_0009.mp4
→ Real frames saved to: /content/drive/MyDrive/Internship2025/Mediapipe_output_face_frame/Celeb_synthesis_mediapipe_output/frames_synthesis_mediapipe/id29_id21_0009
→ Cropped faces saved to: /content/drive/MyDrive/Internship2025/Mediapipe_output_face_frame/Celeb_synthesis_mediapipe_output/faces_synthesis_mediapipe/id29_id21_0009
Processing video: id28_id35_0003.mp4
Finished processing: id28_id35_0003.mp4
→ Real frames saved to: /content/drive/MyDrive/Internship2025/Mediapipe_output_face_frame/Celeb_synthesis_m

KeyboardInterrupt: 